In [1]:
import pandas as pd
import dash
from dash import dcc, html
import dash_table
import plotly.express as px

C:\Users\walje\AppData\Local\Temp\ipykernel_38692\3497302629.py:4: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


In [2]:
df = pd.read_csv('cfb_line_stats.csv')

ranking_df = pd.DataFrame({'Ranking': range(1, 126)})

In [3]:
app = dash.Dash(__name__)

In [4]:
# conference dropdown
conference_dropdown = dcc.Dropdown(
    id='conference-dropdown',
    options=[{'label': conference, 'value': conference} for conference in df['Conference'].unique()] + [{'label': 'All', 'value': 'All'}],
    value='All',  # Default selection
    clearable=False
)

# visualization dropdown 
metric_dropdown = dcc.Dropdown(
    id='metric-dropdown',
    options=[{'label': col, 'value': col} for col in df.columns if col not in ['Team', 'Conference', 'Ranking']],
    value='plays',  # Default selection metric
    clearable=False
)

In [5]:
@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('conference-dropdown', 'value'),
     dash.dependencies.Input('metric-dropdown', 'value')]
)
def update_graph(selected_conference, selected_metric):

# conference filter
    if selected_conference == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['Conference'] == selected_conference]

# bar plot
    fig = px.bar(filtered_df.sort_values(by=selected_metric, ascending=False), 
                 x='Team', y=selected_metric, title=f'{selected_metric} by Team')

    return fig

@app.callback(
    dash.dependencies.Output('data-table', 'data'),
    [dash.dependencies.Input('conference-dropdown', 'value'),
     dash.dependencies.Input('data-table', 'sort_by'),
     dash.dependencies.Input('data-table', 'filter_query')]
)
def update_table(selected_conference, sort_by, filter_query):
    filtered_df = df.copy()

# apply conference filtering
    if selected_conference != 'All':
        filtered_df = filtered_df[filtered_df['Conference'] == selected_conference]

# apply filtering query (if necessary)
    if filter_query:
        filtered_df = filtered_df.query(filter_query)

# sort by
    if sort_by:
        filtered_df = filtered_df.sort_values(
            by=[col['column_id'] for col in sort_by],
            ascending=[col['direction'] == 'asc' for col in sort_by]
        )

    return filtered_df.to_dict('records')

In [6]:
# app layout
app.layout = html.Div([
    html.H1("CFB Line Stats"),
    conference_dropdown,
    metric_dropdown, 
    dcc.Graph(id='graph'),  
    html.Div([
        dash_table.DataTable(
            id='ranking-table',
            columns=[{'name': 'Ranking', 'id': 'Ranking'}],  
            data=ranking_df.to_dict('records'),  
            sort_action='none',  
            filter_action='none', 
            page_size=125,
            style_table={'width': '50px', 'overflowX': 'hidden'},
            style_cell={'textAlign': 'center'},
        ),
        dash_table.DataTable(
            id='data-table',
            columns=[{'name': col, 'id': col, 'sortable': True} for col in df.columns],
            data=df.to_dict('records'), 
            sort_action='native',
            filter_action='none', 
            page_size=125,  
            style_table={'overflowX': 'auto'},
            style_cell={'padding': '5px'},
        ),
    ], style={'display': 'flex', 'alignItems': 'start'}),
])

In [7]:
app.run_server(debug=False, use_reloader=False, port=8070)